IMPORTING LIBRARIES

In [0]:
import pandas as pd
import numpy as np
import re
from nltk.stem.porter import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

In [10]:
data = pd.read_csv('train.csv',encoding='latin-1',header=None)
data.columns = ["Polarity", "Id", "Date", "Query","User","Text"]
data.head(25)

,Polarity,Id,Date,Query,User,Text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [11]:
data.Polarity.value_counts()

4    800000
0    800000
Name: Polarity, dtype: int64

In [12]:
data.isnull().sum()

Polarity    0
Id          0
Date        0
Query       0
User        0
Text        0
dtype: int64

**DATA CLEANING**

In [13]:
data.drop(['Id','Date','Query','User'],axis=1,inplace=True)
data.head(25)

,Polarity,Text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew
6,0,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,@Tatiana_K nope they didn't have it
9,0,@twittera que me muera ?


In [0]:
def remove(text):
    new = re.sub(r'@[A-Za-z0-9_]+','',text)
    neww = re.sub(r'https?://[^ ]+','',new)
    return neww

data['Clean_text'] = data.apply(lambda x: remove(x['Text']),axis=1)
data['Clean_text'] = data['Clean_text'].str.replace('[^A-Za-z#]',' ')
#data['Clean_text'] = data['Clean_text'].apply(lambda x: ' '.join([i for i in x.split() if len(i)>3]) )   #Not using small words like at,to,the,etc

In [15]:
tokens = data['Clean_text'].apply(lambda x:x.split())
#tokens
stemmer = PorterStemmer()
tokens = tokens.apply(lambda x: [stemmer.stem(i) for i in x])
#tokens.head()
data['Clean_text'] = tokens.apply(lambda x: ' '.join(x))
data

,Polarity,Text,Clean_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww that s a bummer you shoulda got david car...
1,0,is upset that he can't update his Facebook by ...,is upset that he can t updat hi facebook by te...
2,0,@Kenichan I dived many times for the ball. Man...,I dive mani time for the ball manag to save th...
3,0,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire
4,0,"@nationwideclass no, it's not behaving at all....",no it s not behav at all i m mad whi am i here...
5,0,@Kwesidei not the whole crew,not the whole crew
6,0,Need a hug,need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...,hey long time no see ye rain a bit onli a bit ...
8,0,@Tatiana_K nope they didn't have it,nope they didn t have it
9,0,@twittera que me muera ?,que me muera


In [16]:
data.isnull().sum()
data.head(25)

,Polarity,Text,Clean_text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww that s a bummer you shoulda got david car...
1,0,is upset that he can't update his Facebook by ...,is upset that he can t updat hi facebook by te...
2,0,@Kenichan I dived many times for the ball. Man...,I dive mani time for the ball manag to save th...
3,0,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire
4,0,"@nationwideclass no, it's not behaving at all....",no it s not behav at all i m mad whi am i here...
5,0,@Kwesidei not the whole crew,not the whole crew
6,0,Need a hug,need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...,hey long time no see ye rain a bit onli a bit ...
8,0,@Tatiana_K nope they didn't have it,nope they didn t have it
9,0,@twittera que me muera ?,que me muera


In [17]:
data['Length'] = data.apply(lambda x : len(x['Clean_text']),axis=1)
data

,Polarity,Text,Clean_text,Length
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",awww that s a bummer you shoulda got david car...,71
1,0,is upset that he can't update his Facebook by ...,is upset that he can t updat hi facebook by te...,100
2,0,@Kenichan I dived many times for the ball. Man...,I dive mani time for the ball manag to save th...,68
3,0,my whole body feels itchy and like its on fire,my whole bodi feel itchi and like it on fire,44
4,0,"@nationwideclass no, it's not behaving at all....",no it s not behav at all i m mad whi am i here...,84
5,0,@Kwesidei not the whole crew,not the whole crew,18
6,0,Need a hug,need a hug,10
7,0,@LOLTrish hey long time no see! Yes.. Rains a...,hey long time no see ye rain a bit onli a bit ...,74
8,0,@Tatiana_K nope they didn't have it,nope they didn t have it,24
9,0,@twittera que me muera ?,que me muera,12


**MODELLING**

In [18]:
y = data['Polarity']
data.drop('Polarity',axis=1,inplace=True)
x = data['Clean_text']
X_train, X_test, y_train, y_test = train_test_split(x,y,random_state=0)
vect = TfidfVectorizer(min_df=5,ngram_range=[1,2]).fit(X_train)
X_train_vectorized = vect.transform(X_train)
len(vect.get_feature_names())

281181

In [19]:
#LOGISTIC REGRESSION
model = LogisticRegression().fit(X_train_vectorized,y_train)
pred = model.predict(vect.transform(X_test))
accuracy_score(y_test,pred)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8221375

In [20]:
#MULTINOMIAL NAIVE BAYES
clf = MultinomialNB(alpha=0.1).fit(X_train_vectorized,y_train)
pred1 = clf.predict(vect.transform(X_test))
accuracy_score(y_test,pred1)

0.7973575

In [21]:
print(model.predict(vect.transform(['very nicee'])))

[4]


In [23]:
target_names = ['Bad', 'Good']
print(classification_report(y_test, pred1, target_names=target_names))

              precision    recall  f1-score   support

         Bad       0.79      0.81      0.80    199734
        Good       0.80      0.79      0.80    200266

    accuracy                           0.80    400000
   macro avg       0.80      0.80      0.80    400000
weighted avg       0.80      0.80      0.80    400000



In [24]:
cm = confusion_matrix(y_test, pred)
print(cm)

[[162347  37387]
 [ 33758 166508]]
